In [1]:
MAKE_SUBMISSION = True          # Generate output file.
CV_ONLY = False                 # Do validation only; do not generate predicitons.
FIT_FULL_TRAIN_SET = True       # Fit model to full training set after doing validation.
FIT_2017_TRAIN_SET = False      # Use 2017 training data for full fit (no leak correction)
FIT_COMBINED_TRAIN_SET = True   # Fit combined 2016-2017 training set
USE_SEASONAL_FEATURES = True
VAL_SPLIT_DATE = '2016-09-15'   # Cutoff date for validation split
LEARNING_RATE = 0.007           # shrinkage rate for boosting roudns
ROUNDS_PER_ETA = 20             # maximum number of boosting rounds times learning rate
OPTIMIZE_FUDGE_FACTOR = False   # Optimize factor by which to multiply predictions.
FUDGE_FACTOR_SCALEDOWN = 0.3    # exponent to reduce optimized fudge factor for prediction

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import datetime as dt
from datetime import datetime
import gc
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg


/home/renmeng/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/renmeng/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
properties16 = pd.read_csv('../data/properties_2016.csv/properties_2016.csv', low_memory = False)
# properties17 = pd.read_csv('../data/properties_2017.csv', low_memory = False)

# Number of properties in the zip
zip_count = properties16['regionidzip'].value_counts().to_dict()
# Number of properties in the city
city_count = properties16['regionidcity'].value_counts().to_dict()
# Median year of construction by neighborhood
medyear = properties16.groupby('regionidneighborhood')['yearbuilt'].aggregate('median').to_dict()
# Mean square feet by neighborhood
meanarea = properties16.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].aggregate('mean').to_dict()
# Neighborhood latitude and longitude
medlat = properties16.groupby('regionidneighborhood')['latitude'].aggregate('median').to_dict()
medlong = properties16.groupby('regionidneighborhood')['longitude'].aggregate('median').to_dict()

train = pd.read_csv("../data/train_2016_v2.csv/train_2016_v2.csv")
for c in properties16.columns:
    properties16[c]=properties16[c].fillna(-1)
    if properties16[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties16[c].values))
        properties16[c] = lbl.transform(list(properties16[c].values))

In [38]:
?LabelEncoder()

In [4]:
train_df = train.merge(properties16, how='left', on='parcelid')
#2016-09-15
select_qtr4 = pd.to_datetime(train_df["transactiondate"]) >= VAL_SPLIT_DATE
if USE_SEASONAL_FEATURES:
    basedate = pd.to_datetime('2015-11-15').toordinal()

In [5]:
# Inputs to features that depend on target variable
# (Ideally these should be recalculated, and the dependent features recalculated,
#  when fitting to the full training set.  But I haven't implemented that yet.)

# Standard deviation of target value for properties in the city/zip/neighborhood
citystd = train_df[~select_qtr4].groupby('regionidcity')['logerror'].aggregate("std").to_dict()
zipstd = train_df[~select_qtr4].groupby('regionidzip')['logerror'].aggregate("std").to_dict()
hoodstd = train_df[~select_qtr4].groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()

In [6]:
train_df = train.merge(properties16, how='left', on='parcelid')
train_df.shape

(90275, 60)

In [7]:
def calculate_features(df):
    # Nikunj's features
    # Number of properties in the zip
    df['N-zip_count'] = df['regionidzip'].map(zip_count)
    # Number of properties in the city
    df['N-city_count'] = df['regionidcity'].map(city_count)
    # Does property have a garage, pool or hot tub and AC?
    df['N-GarPoolAC'] = ((df['garagecarcnt']>0) & \
                         (df['pooltypeid10']>0) & \
                         (df['airconditioningtypeid']!=5))*1 

    # More features
    # Mean square feet of neighborhood properties
    df['mean_area'] = df['regionidneighborhood'].map(meanarea)
    # Median year of construction of neighborhood properties
    df['med_year'] = df['regionidneighborhood'].map(medyear)
    # Neighborhood latitude and longitude
    df['med_lat'] = df['regionidneighborhood'].map(medlat)
    df['med_long'] = df['regionidneighborhood'].map(medlong)

    df['zip_std'] = df['regionidzip'].map(zipstd)
    df['city_std'] = df['regionidcity'].map(citystd)
    df['hood_std'] = df['regionidneighborhood'].map(hoodstd)
    
    if USE_SEASONAL_FEATURES:
        df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
        df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)

In [75]:
tmp = pd.to_datetime('2016-01-01').toordinal()-pd.to_datetime('2015-11-15').toordinal()

In [76]:
tmp * (2 * np.pi/365.25)

0.8085139204310487

In [8]:
dropvars = ['airconditioningtypeid', 'buildingclasstypeid',
            'buildingqualitytypeid', 'regionidcity']
droptrain = ['parcelid', 'logerror', 'transactiondate']
droptest = ['ParcelId']

In [9]:
train_df.columns

Index([u'parcelid', u'logerror', u'transactiondate', u'airconditioningtypeid',
       u'architecturalstyletypeid', u'basementsqft', u'bathroomcnt',
       u'bedroomcnt', u'buildingclasstypeid', u'buildingqualitytypeid',
       u'calculatedbathnbr', u'decktypeid', u'finishedfloor1squarefeet',
       u'calculatedfinishedsquarefeet', u'finishedsquarefeet12',
       u'finishedsquarefeet13', u'finishedsquarefeet15',
       u'finishedsquarefeet50', u'finishedsquarefeet6', u'fips',
       u'fireplacecnt', u'fullbathcnt', u'garagecarcnt', u'garagetotalsqft',
       u'hashottuborspa', u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequart

In [10]:
calculate_features(train_df)

x_valid = train_df.drop(dropvars+droptrain, axis=1)[select_qtr4]
y_valid = train_df["logerror"].values.astype(np.float32)[select_qtr4]

print('Shape full training set: {}'.format(train_df.shape))
print('Dropped vars: {}'.format(len(dropvars+droptrain)))
print('Shape valid X: {}'.format(x_valid.shape))
print('Shape valid y: {}'.format(y_valid.shape))

train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
print('\nFull training set after removing outliers, before dropping vars:')     
print('Shape training set: {}\n'.format(train_df.shape))

if FIT_FULL_TRAIN_SET:
    full_train = train_df.copy()

train_df=train_df[~select_qtr4]
x_train=train_df.drop(dropvars+droptrain, axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)
n_train = x_train.shape[0]
print('Training subset after removing outliers:')     
print('Shape train X: {}'.format(x_train.shape))
print('Shape train y: {}'.format(y_train.shape))

if FIT_FULL_TRAIN_SET:
    x_full = full_train.drop(dropvars+droptrain, axis=1)
    y_full = full_train["logerror"].values.astype(np.float32)
    n_full = x_full.shape[0]
    print('\nFull trainng set:')     
    print('Shape train X: {}'.format(x_train.shape))
    print('Shape train y: {}'.format(y_train.shape))

Shape full training set: (90275, 72)
Dropped vars: 7
Shape valid X: (14304, 65)
Shape valid y: (14304,)

Full training set after removing outliers, before dropping vars:
Shape training set: (88528, 72)

Training subset after removing outliers:
Shape train X: (74478, 65)
Shape train y: (74478,)

Full trainng set:
Shape train X: (74478, 65)
Shape train y: (74478,)


/home/renmeng/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [12]:
if not CV_ONLY:
    # Generate test set data
    
    sample_submission = pd.read_csv('../data/sample_submission.csv', low_memory = False)
    
    # Process properties for 2016
    test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )
    if USE_SEASONAL_FEATURES:
        test_df['transactiondate'] = '2016-10-31'
        droptest += ['transactiondate']
    calculate_features(test_df)
    x_test = test_df.drop(dropvars+droptest, axis=1)
    print('Shape test: {}'.format(x_test.shape))

    # Process properties for 2017
#     for c in properties17.columns:
#         properties17[c]=properties17[c].fillna(-1)
#         if properties17[c].dtype == 'object':
#             lbl = LabelEncoder()
#             lbl.fit(list(properties17[c].values))
#             properties17[c] = lbl.transform(list(properties17[c].values))
#     zip_count = properties17['regionidzip'].value_counts().to_dict()
#     city_count = properties17['regionidcity'].value_counts().to_dict()
#     medyear = properties17.groupby('regionidneighborhood')['yearbuilt'].aggregate('median').to_dict()
#     meanarea = properties17.groupby('regionidneighborhood')['calculatedfinishedsquarefeet'].aggregate('mean').to_dict()
#     medlat = properties17.groupby('regionidneighborhood')['latitude'].aggregate('median').to_dict()
#     medlong = properties17.groupby('regionidneighborhood')['longitude'].aggregate('median').to_dict()

#     test_df = pd.merge( sample_submission[['ParcelId']], 
#                         properties17.rename(columns = {'parcelid': 'ParcelId'}), 
#                         how = 'left', on = 'ParcelId' )
#     if USE_SEASONAL_FEATURES:
#         test_df['transactiondate'] = '2017-10-31'
#     calculate_features(test_df)
#     x_test17 = test_df.drop(dropvars+droptest, axis=1)

    del test_df

Shape test: (2985217, 65)


In [11]:
# x_test17.shape

In [13]:
xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': LEARNING_RATE,
    'max_depth': 7, 
    'subsample': 0.6,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 5.0,
    'alpha': 0.65,
    'colsample_bytree': 0.5,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dvalid_x = xgb.DMatrix(x_valid)
dvalid_xy = xgb.DMatrix(x_valid, y_valid)
if not CV_ONLY:
    dtest = xgb.DMatrix(x_test)
#     dtest17 = xgb.DMatrix(x_test17)
#     del x_test

In [14]:
num_boost_rounds = round( ROUNDS_PER_ETA / xgb_params['eta'] )
early_stopping_rounds = round( num_boost_rounds / 20 )
print('Boosting rounds: {}'.format(num_boost_rounds))
print('Early stoping rounds: {}'.format(early_stopping_rounds))

Boosting rounds: 2857.0
Early stoping rounds: 143.0


In [15]:
num_boost_rounds

2857.0

In [16]:
evals = [(dtrain,'train'),(dvalid_xy,'eval')]
model = xgb.train(xgb_params, dtrain, num_boost_round=2875,
                  evals=evals, early_stopping_rounds=early_stopping_rounds, 
                  verbose_eval=100)

[0]	train-mae:0.053445	eval-mae:0.065273
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 143.0 rounds.
[100]	train-mae:0.052722	eval-mae:0.064775
[200]	train-mae:0.052269	eval-mae:0.064542
[300]	train-mae:0.051934	eval-mae:0.064416
[400]	train-mae:0.051687	eval-mae:0.064353
[500]	train-mae:0.051454	eval-mae:0.064314
[600]	train-mae:0.051238	eval-mae:0.064283
[700]	train-mae:0.051049	eval-mae:0.064273
[800]	train-mae:0.050861	eval-mae:0.064261
[900]	train-mae:0.050691	eval-mae:0.064266
Stopping. Best iteration:
[804]	train-mae:0.050853	eval-mae:0.064258



In [95]:
?model.best_ntree_limit

In [96]:
valid_pred = model.predict(dvalid_x, ntree_limit=model.best_ntree_limit)
print( "XGBoost validation set predictions:" )
print( pd.DataFrame(valid_pred).head() )
print("\nMean absolute validation error:")
mean_absolute_error(y_valid, valid_pred)


XGBoost validation set predictions:
          0
0  0.001607
1  0.022175
2  0.020656
3  0.012385
4  0.019222

Mean absolute validation error:


0.064258203

In [ ]:
submit = pd.read_csv('../data/sample_submission.csv')

In [107]:
x_test.shape

(2985217, 65)

In [108]:
test_pred = model.predict(dtest, ntree_limit=model.best_ntree_limit)

print test_pred.shape
print submit.shape
for c in submit.columns:
    if c != 'ParcelId':
        submit[c] = test_pred
print submit.head(100)
submit.to_csv('../data/feature_65.csv', index=False)

In [110]:
?QuantReg

In [112]:
if OPTIMIZE_FUDGE_FACTOR:
    mod = QuantReg(y_valid, valid_pred)
    res = mod.fit(q=.5)
    print("\nLAD Fit for Fudge Factor:")
    print(res.summary())

    fudge = res.params[0]
    print("Optimized fudge factor:", fudge)
    print("\nMean absolute validation error with optimized fudge factor: ")
    print(mean_absolute_error(y_valid, fudge*valid_pred))

    fudge **= FUDGE_FACTOR_SCALEDOWN
    print("Scaled down fudge factor:", fudge)
    print("\nMean absolute validation error with scaled down fudge factor: ")
    print(mean_absolute_error(y_valid, fudge*valid_pred))
else:
    fudge=1.0

In [115]:
if FIT_FULL_TRAIN_SET and not CV_ONLY:
    if not FIT_COMBINED_TRAIN_SET:
        # Merge 2016 and 2017 data sets
        train16 = pd.read_csv('../input/train_2016_v2.csv')
        train17 = pd.read_csv('../input/train_2017.csv')
        train16 = pd.merge(train16, properties16, how = 'left', on = 'parcelid')
        train17 = pd.merge(train17, properties17, how = 'left', on = 'parcelid')
        train17[['structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxvaluedollarcnt', 'taxamount']] = np.nan
        train_df = pd.concat([train16, train17], axis = 0)
        # Generate features
        citystd = train_df.groupby('regionidcity')['logerror'].aggregate("std").to_dict()
        zipstd = train_df.groupby('regionidzip')['logerror'].aggregate("std").to_dict()
        hoodstd = train_df.groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()
        calculate_features(train_df)
        # Remove outliers
        train_df=train_df[ train_df.logerror > -0.4 ]
        train_df=train_df[ train_df.logerror < 0.419 ]
        # Create final training data sets
        x_full = train_df.drop(dropvars+droptrain, axis=1)
        y_full = train_df["logerror"].values.astype(np.float32)
        n_full = x_full.shape[0]     
    elif FIT_2017_TRAIN_SET:
        train = pd.read_csv('../data/train_2017.csv')
        train_df = train.merge(properties17, how='left', on='parcelid')
        # Generate features
        citystd = train_df.groupby('regionidcity')['logerror'].aggregate("std").to_dict()
        zipstd = train_df.groupby('regionidzip')['logerror'].aggregate("std").to_dict()
        hoodstd = train_df.groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()
        calculate_features(train_df)
        # Remove outliers
        train_df=train_df[ train_df.logerror > -0.4 ]
        train_df=train_df[ train_df.logerror < 0.419 ]
        # Create final training data sets
        x_full = train_df.drop(dropvars+droptrain, axis=1)
        y_full = train_df["logerror"].values.astype(np.float32)
        n_full = x_full.shape[0]     
    dtrain = xgb.DMatrix(x_full, y_full)
    num_boost_rounds = int(model.best_ntree_limit*n_full/n_train)
    full_model = xgb.train(xgb_params, dtrain, num_boost_round=num_boost_rounds, 
                           evals=[(dtrain,'train')], verbose_eval=10)

[0]	train-mae:0.053266
[10]	train-mae:0.053176
[20]	train-mae:0.053092
[30]	train-mae:0.053007
[40]	train-mae:0.052932
[50]	train-mae:0.052855
[60]	train-mae:0.052788
[70]	train-mae:0.052725
[80]	train-mae:0.052671
[90]	train-mae:0.052615
[100]	train-mae:0.052562
[110]	train-mae:0.05251
[120]	train-mae:0.052461
[130]	train-mae:0.052414
[140]	train-mae:0.052365
[150]	train-mae:0.052319
[160]	train-mae:0.05228
[170]	train-mae:0.052237
[180]	train-mae:0.052199
[190]	train-mae:0.052164
[200]	train-mae:0.052128
[210]	train-mae:0.052095
[220]	train-mae:0.052058
[230]	train-mae:0.052024
[240]	train-mae:0.051993
[250]	train-mae:0.051962
[260]	train-mae:0.051933
[270]	train-mae:0.051905
[280]	train-mae:0.051878
[290]	train-mae:0.051849
[300]	train-mae:0.051818
[310]	train-mae:0.051792
[320]	train-mae:0.051766
[330]	train-mae:0.051741
[340]	train-mae:0.051716
[350]	train-mae:0.051689
[360]	train-mae:0.051663
[370]	train-mae:0.051638
[380]	train-mae:0.051614
[390]	train-mae:0.051592
[400]	train-m

In [ ]:
del properties16
del properties17
gc.collect()

In [116]:
fudge

1.0

In [117]:
if not CV_ONLY:
    if FIT_FULL_TRAIN_SET:
        pred = fudge*full_model.predict(dtest)
        pred17 = fudge*full_model.predict(dtest17)
    else:
        pred = fudge*model.predict(dtest, ntree_limit=model.best_ntree_limit)
        pred17 = fudge*model.predict(dtest17, ntree_limit=model.best_ntree_limit)
        
    print( "XGBoost test set predictions for 2016:" )
    print( pd.DataFrame(pred).head() )
    print( "XGBoost test set predictions for 2017:" )
    print( pd.DataFrame(pred17).head() )    

XGBoost test set predictions for 2016:
          0
0  0.002543
1 -0.006614
2  0.025026
3  0.044890
4 -0.002124
XGBoost test set predictions for 2017:
          0
0  0.002543
1 -0.006614
2  0.031241
3  0.045232
4 -0.001447


NameError: name 'test' is not defined

In [ ]:

# Process properties for 2016
# test_df = pd.merge( sample_submission[['ParcelId']], 
#                     properties16.rename(columns = {'parcelid': 'ParcelId'}), 
#                     how = 'left', on = 'ParcelId' )

In [28]:
sample_submission = pd.read_csv('../data/sample_submission.csv', low_memory = False)

test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )
test_df['transactiondate'] = '2016-10-31'
calculate_features(test_df)
x_test = test_df.drop(dropvars+droptest, axis=1)
print('Shape test: {}'.format(x_test.shape))
dtest = xgb.DMatrix(x_test)
test_10 = model.predict(dtest, ntree_limit=model.best_ntree_limit)

###
test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )
test_df['transactiondate'] = '2016-11-30'
calculate_features(test_df)
x_test = test_df.drop(dropvars+droptest, axis=1)
print('Shape test: {}'.format(x_test.shape))
dtest = xgb.DMatrix(x_test)
test_11 = model.predict(dtest, ntree_limit=model.best_ntree_limit)

###
test_df = pd.merge( sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )
test_df['transactiondate'] = '2016-12-31'
calculate_features(test_df)
x_test = test_df.drop(dropvars+droptest, axis=1)
print('Shape test: {}'.format(x_test.shape))
dtest = xgb.DMatrix(x_test)
test_12 = model.predict(dtest, ntree_limit=model.best_ntree_limit)



output = pd.DataFrame({'ParcelId': sample_submission['ParcelId'].astype(np.int32),
           '201610': test_10, '201611': test_11, '201612': test_12,
           '201710': test_10, '201711': test_11, '201712': test_12})
output.to_csv('../data/64258_with_time.csv',index=False)

Shape test: (2985217, 65)
Shape test: (2985217, 65)
Shape test: (2985217, 65)


,ParcelId,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,N-GarPoolAC,mean_area,med_year,med_lat,med_long,zip_std,city_std,hood_std,cos_season,sin_season
0,10754147,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,0,NaN,NaN,NaN,NaN,0.096225,0.104574,0.154446,0.970105,-0.242687
1,10759547,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,0,NaN,NaN,NaN,NaN,0.096225,0.104574,0.154446,0.970105,-0.242687
2,10843547,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,...,0,NaN,NaN,NaN,NaN,0.143986,0.140596,0.154446,0.970105,-0.242687
3,10859147,-1.0,-1.0,-1.0,0.0,0.0,3.0,7.0,-1.0,-1.0,...,0,1989.878889,1955.0,34156200.0,-118444765.0,0.128849,0.195053,0.151713,0.970105,-0.242687
4,10879947,-1.0,-1.0,-1.0,0.0,0.0,4.0,-1.0,-1.0,-1.0,...,0,1530.869588,1947.0,34179126.0,-118374599.0,0.143689,0.195053,0.221739,0.970105,-0.242687


In [120]:
print("Mean absolute validation error without fudge factor: ", )
print( mean_absolute_error(y_valid, valid_pred) )
if OPTIMIZE_FUDGE_FACTOR:
    print("Mean absolute validation error with fudge factor:")
    print( mean_absolute_error(y_valid, fudge*valid_pred) )

('Mean absolute validation error without fudge factor: ',)
0.0642582
